Nama: Janice Ashley Kurniawan

NIM: 2602203794

UAS Model Deployment

In [1]:
# import library
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
import pickle

### Load Dataset

In [3]:
# read data
data = pd.read_csv('../dataset/data_1D.csv')
data

,Unnamed: 0,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,0,83,retired,divorced,basic.4y,no,no,no,cellular,nov,tue,242.0,1,3,3,success,yes
1,1,32,services,married,high.school,no,no,no,telephone,may,tue,190.0,3,999,0,nonexistent,no
2,2,31,admin.,single,university.degree,no,yes,no,telephone,may,wed,325.0,2,999,0,nonexistent,no
3,3,33,admin.,single,university.degree,no,yes,no,telephone,aug,thu,158.0,1,999,0,nonexistent,no
4,4,39,services,married,high.school,no,yes,no,telephone,jul,mon,158.0,1,999,0,nonexistent,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8233,8233,33,technician,single,university.degree,no,no,yes,telephone,may,mon,471.0,1,999,0,nonexistent,no
8234,8234,47,services,married,high.school,no,no,no,telephone,may,tue,98.0,1,999,0,nonexistent,no
8235,8235,24,services,single,basic.9y,no,yes,yes,cellular,may,fri,68.0,4,999,1,failure,no
8236,8236,39,entrepreneur,married,high.school,no,no,no,cellular,nov,thu,150.0,1,999,0,nonexistent,no


### Data Overview

In [4]:
data.shape

(8238, 17)

In [5]:
data.columns

Index(['Unnamed: 0', 'age', 'job', 'marital', 'education', 'default',
       'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration',
       'campaign', 'pdays', 'previous', 'poutcome', 'y'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8238 entries, 0 to 8237
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   8238 non-null   int64  
 1   age          8238 non-null   int64  
 2   job          8230 non-null   object 
 3   marital      8238 non-null   object 
 4   education    8238 non-null   object 
 5   default      8238 non-null   object 
 6   housing      8238 non-null   object 
 7   loan         8238 non-null   object 
 8   contact      8238 non-null   object 
 9   month        8238 non-null   object 
 10  day_of_week  8238 non-null   object 
 11  duration     8226 non-null   float64
 12  campaign     8238 non-null   int64  
 13  pdays        8238 non-null   int64  
 14  previous     8238 non-null   int64  
 15  poutcome     8238 non-null   object 
 16  y            8238 non-null   object 
dtypes: float64(1), int64(5), object(11)
memory usage: 1.1+ MB


### Data Preprocessing

In [7]:
for i in data.columns:
  print(data[i].value_counts())
  print()

Unnamed: 0
0       1
5534    1
5500    1
5499    1
5498    1
       ..
2741    1
2740    1
2739    1
2738    1
8237    1
Name: count, Length: 8238, dtype: int64

age
31    401
33    383
32    371
36    357
35    355
     ... 
67      2
86      2
85      1
89      1
17      1
Name: count, Length: 72, dtype: int64

job
admin.           2033
blue-collar      1870
technician       1372
services          822
management        546
retired           366
entrepreneur      294
self-employed     263
housemaid         219
unemployed        196
student           184
unknown            65
Name: count, dtype: int64

marital
married     4994
single      2338
divorced     892
unknown       14
Name: count, dtype: int64

education
university.degree      2379
high.school            1915
basic.9y               1243
professional.course    1043
basic.4y                817
basic.6y                486
unknown                 349
illiterate                6
Name: count, dtype: int64

default
no         6491
un

drop 'Unnamed: 0' column

In [8]:
data.drop('Unnamed: 0', axis=1, inplace=True)

checking for missing values

In [9]:
data.isna().sum()

age             0
job             8
marital         0
education       0
default         0
housing         0
loan            0
contact         0
month           0
day_of_week     0
duration       12
campaign        0
pdays           0
previous        0
poutcome        0
y               0
dtype: int64

In [10]:
data.dropna(inplace=True)
data.isna().sum()

age            0
job            0
marital        0
education      0
default        0
housing        0
loan           0
contact        0
month          0
day_of_week    0
duration       0
campaign       0
pdays          0
previous       0
poutcome       0
y              0
dtype: int64

checking for duplicated rows

In [11]:
data.duplicated().sum()

0

identifying columns as numerical and categorical

In [12]:
num_cols = []
cat_cols = []

for i in data.columns:
    if data[i].dtype == 'object':
        cat_cols.append(i)
    elif "int" in str(data[i].dtype) or "float" in str(data[i].dtype):
        num_cols.append(i)

print(num_cols)
print(cat_cols)

['age', 'duration', 'campaign', 'pdays', 'previous']
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y']


In [13]:
data[num_cols].describe()

,age,duration,campaign,pdays,previous
count,8218.000000,8218.000000,8218.000000,8218.000000,8218.000000
mean,39.929058,258.233025,2.565344,963.819421,0.169993
std,10.517773,266.848341,2.794260,183.628538,0.493048
min,17.000000,4.000000,1.000000,0.000000,0.000000
25%,32.000000,102.000000,1.000000,999.000000,0.000000
50%,38.000000,180.000000,2.000000,999.000000,0.000000
75%,47.000000,320.000000,3.000000,999.000000,0.000000
max,89.000000,4918.000000,43.000000,999.000000,6.000000


encoding categorical columns

In [14]:
encoder = LabelEncoder()
for col in cat_cols:
    data[col] = encoder.fit_transform(data[col])

### Preprocessed Data

In [15]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,83,5,0,0,0,0,0,0,7,3,242.0,1,3,3,2,1
1,32,7,1,3,0,0,0,1,6,3,190.0,3,999,0,1,0
2,31,0,2,6,0,2,0,1,6,4,325.0,2,999,0,1,0
3,33,0,2,6,0,2,0,1,1,2,158.0,1,999,0,1,0
4,39,7,1,3,0,2,0,1,3,1,158.0,1,999,0,1,0


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8218 entries, 0 to 8237
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          8218 non-null   int64  
 1   job          8218 non-null   int64  
 2   marital      8218 non-null   int64  
 3   education    8218 non-null   int64  
 4   default      8218 non-null   int64  
 5   housing      8218 non-null   int64  
 6   loan         8218 non-null   int64  
 7   contact      8218 non-null   int64  
 8   month        8218 non-null   int64  
 9   day_of_week  8218 non-null   int64  
 10  duration     8218 non-null   float64
 11  campaign     8218 non-null   int64  
 12  pdays        8218 non-null   int64  
 13  previous     8218 non-null   int64  
 14  poutcome     8218 non-null   int64  
 15  y            8218 non-null   int64  
dtypes: float64(1), int64(15)
memory usage: 1.1 MB


Data Splitting (80% Train Data and 20% Test Data)

In [17]:
X = data.drop('y', axis=1)
y = data['y']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train Data:", len(X_train))
print("Test Data:", len(X_test))

Train Data: 6574
Test Data: 1644


### Logistic Regression Model

In [19]:
# initialize and train Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
# predictions and evaluation for Logistic Regression
y_pred_logreg = logreg.predict(X_test)
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_logreg)}")
print("Logistic Regression Report:\n", classification_report(y_test, y_pred_logreg))
print("Logistic Regression ROC-AUC Score:", roc_auc_score(y_test, y_pred_logreg))

Logistic Regression Accuracy: 0.9051094890510949
Logistic Regression Report:
               precision    recall  f1-score   support

           0       0.91      0.99      0.95      1449
           1       0.77      0.28      0.41       195

    accuracy                           0.91      1644
   macro avg       0.84      0.64      0.68      1644
weighted avg       0.89      0.91      0.88      1644

Logistic Regression ROC-AUC Score: 0.6355045920263311


### Random Forest Model

In [21]:
# initialize and train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [22]:
# predictions and evaluation for Random Forest
y_pred_rf = rf.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf)}")
print("Random Forest Report:\n", classification_report(y_test, y_pred_rf))
print("Random Forest ROC-AUC Score:", roc_auc_score(y_test, y_pred_rf))

Random Forest Accuracy: 0.9051094890510949
Random Forest Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95      1449
           1       0.72      0.32      0.45       195

    accuracy                           0.91      1644
   macro avg       0.82      0.65      0.70      1644
weighted avg       0.89      0.91      0.89      1644

Random Forest ROC-AUC Score: 0.6532568880394967


### Saving Random Forest Model to File Using Pickle

In [23]:
with open('../model/random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf, file)

### Link Video

https://drive.google.com/drive/folders/1tf_z01SNC-OJeW838_6lQsdx4NB60mNd?usp=sharing